In [1]:
#Import packages
import boto3
import json
import numpy as np
from numpy import argmax
import pyspark.sql.functions as F
import snowflake.connector
import pandas as pd
import spacy
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from urllib.parse import urlparse
from langdetect import detect
from textblob import TextBlob
#RegEx
import regex as re

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1616676767755_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
sm_client = boto3.client('secretsmanager',region_name='us-west-2')
secret_value = sm_client.get_secret_value(SecretId = 'emr_spark_qa')
username = json.loads(secret_value['SecretString'])['emr_spark_username']
password = json.loads(secret_value['SecretString'])['emr_spark_password']
#print(username)
#print(password)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
#PYTHON connection info
conn = snowflake.connector.connect(
    account='powerreviews',
    database='analytics_prod',
    warehouse='LOWER_ENV_WAREHOUSE',
    user=username,
    password=password
    )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [107]:
#Query some reviews
query_1 = "select distinct r.ugc_id, r.review_headline, r.review_comments, substr(r.locale,1,2) as lang, r.review_length, \
case when contains(r.observations, 'FL')  then 1 else 0 end as FL \
from analytics.review as r \
join analytics.merchant_group as mg \
on r.client_id = mg.client_id \
where contains(r.observations, 'PM') \
and not contains(r.observations, 'IA') and not contains(r.observations, 'IR') \
and mg.usage_type != 'Test' \
and r.created_date > '2020-10-01 11:00' \
and r.created_date < '2021-01-01 11:00' \
and review_length < 20 \
and contains(r.observations, 'FL') \
"

#193267606, 324912854
#and r.created_date > '2020-01-01' \

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [108]:
reviews = pd.read_sql(query_1, conn)
reviews.head()
reviews.info()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1314 entries, 0 to 1313
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   UGC_ID           1314 non-null   int64 
 1   REVIEW_HEADLINE  1314 non-null   object
 2   REVIEW_COMMENTS  1314 non-null   object
 3   LANG             1314 non-null   object
 4   REVIEW_LENGTH    1314 non-null   int64 
 5   FL               1314 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 61.7+ KB

In [109]:
#Need to add "word" in front bc otherwise reviews with just emojis, symbols, #s, etc will not work in langdetect package
#Throws this error if no words: "langdetect.lang_detect_exception.LangDetectException: No features in text."
reviews['ALL_TEXT'] = reviews['REVIEW_HEADLINE'] + ' ' + reviews['REVIEW_COMMENTS']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [110]:
from google_trans_new import google_translator  
  
#detector = google_translator()  
#detect_result = detector.detect('this is excellent')
#print(detect_result[0])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [111]:
#import time

#print (time.ctime())
#print (time.ctime())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
from google_trans_new import google_translator  
  
detector = google_translator()  
detect_result = detector.detect('this is excellent')
print(detect_result)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'google_translator' object has no attribute 'translator'
Traceback (most recent call last):
AttributeError: 'google_translator' object has no attribute 'translator'



In [5]:
from googletrans import Translator
translator = Translator()
langs = translator.detect(['한국어', '日本語', 'English', 'le français'])
for lang in langs:
    print(lang.lang, lang.confidence)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'NoneType' object has no attribute 'group'
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/googletrans/client.py", line 251, in detect
    lang = self.detect(item)
  File "/usr/local/lib/python3.7/site-packages/googletrans/client.py", line 255, in detect
    data = self._translate(text, 'en', 'auto', kwargs)
  File "/usr/local/lib/python3.7/site-packages/googletrans/client.py", line 78, in _translate
    token = self.token_acquirer.do(text)
  File "/usr/local/lib/python3.7/site-packages/googletrans/gtoken.py", line 194, in do
    self._update()
  File "/usr/local/lib/python3.7/site-packages/googletrans/gtoken.py", line 62, in _update
    code = self.RE_TKK.search(r.text).group(1).replace('var ', '')
AttributeError: 'NoneType' object has no attribute 'group'



In [112]:

reviews['ALL_TEXT']=reviews['ALL_TEXT'].apply(str)
reviews['PRED_LANG'] = reviews['ALL_TEXT'].apply(lambda x: google_translator().detect(x)[0])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

429 (Too Many Requests) from TTS API. Probable cause: Unknown
Traceback (most recent call last):
  File "/usr/local/lib64/python3.7/site-packages/pandas/core/series.py", line 4138, in apply
    mapped = lib.map_infer(values, f, convert=convert_dtype)
  File "pandas/_libs/lib.pyx", line 2467, in pandas._libs.lib.map_infer
  File "<stdin>", line 3, in <lambda>
  File "/usr/local/lib/python3.7/site-packages/google_trans_new/google_trans_new.py", line 247, in detect
    raise google_new_transError(tts=self, response=r)
google_trans_new.google_trans_new.google_new_transError: 429 (Too Many Requests) from TTS API. Probable cause: Unknown



In [113]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

reviews.head(25)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

       UGC_ID               REVIEW_HEADLINE      REVIEW_COMMENTS LANG  \
0   304789521  Producto de compra inmediata  Great product        en    
1   308926004  Si lo compraría de nuevo.     Muy fasil de usar.   en    
2   319828692  Muy bueno                     Muy buenos zapatos   en    
3   315254774  ممتاز                         ممتاز جدا            en    
4   318095277  Buenisimo                     Esta bien            en    
5   321515119  Lo volveria a comprar         Muy bueno            en    
6   308797051  Si                            El mejor             en    
7   308796996  Si                            Barato               en    
8   307387212  que se repita                 que se repita        en    
9   319285149  También                       Esta bien            en    
10  314432978  Seguro que sí                 Buena marca          en    
11  320946284  EXCELENTE                     ENSALADAS Y CREMAS   en    
12  308752702  Muy recomendable              Son mu

In [114]:
reviews['model_FL'] = 0

reviews.loc[reviews['LANG'] != reviews['PRED_LANG'] , 'model_FL'] = 1

#Our setup may have accidentally used "jp" as well as "ja" for Japenese
#"ja" is official code but mark "jp" as ok too
reviews.loc[ (reviews['LANG']=='jp') & (reviews['PRED_LANG']=='ja') , 'model_FL'] = 0

#'ca' is catalan which is close to spanish
reviews.loc[ (reviews['LANG']=='es') & (reviews['PRED_LANG']=='ca') , 'model_FL'] = 0

reviews.loc[ (reviews['LANG']=='zh') & (reviews['PRED_LANG']=='zh-CN') , 'model_FL'] = 0

#reviews.loc[reviews['REVIEW_LENGTH'] < 40 , 'model_FL'] = 0

#reviews.loc[ (reviews['LANG']=='en') & ((reviews['firstword_eng']==True) | (reviews['secondword_eng']==True)) , 'model_FL'] = 0

#reviews.loc[reviews['PRED_LANG'] == 'UNKNOWN' , 'model_FL'] = 0

#reviews.loc[reviews['PRED_LANG'] == 'af' , 'model_FL'] = 0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'PRED_LANG'
Traceback (most recent call last):
  File "/usr/local/lib64/python3.7/site-packages/pandas/core/frame.py", line 3024, in __getitem__
    indexer = self.columns.get_loc(key)
  File "/usr/local/lib64/python3.7/site-packages/pandas/core/indexes/base.py", line 3082, in get_loc
    raise KeyError(key) from err
KeyError: 'PRED_LANG'



In [115]:
#reviews.head(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [116]:
FP = reviews[(reviews['FL'] == 0) & (reviews['model_FL'] == 1)]
FN = reviews[(reviews['FL'] == 1) & (reviews['model_FL'] == 0)]
TP = reviews[(reviews['FL'] == 1) & (reviews['model_FL'] == 1)]
TN = reviews[(reviews['FL'] == 0) & (reviews['model_FL'] == 0)]


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [117]:

FP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [118]:

FN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1314

In [119]:

TP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [120]:
TN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [121]:
FN.head(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

       UGC_ID                                   REVIEW_HEADLINE  \
0   304789521  Producto de compra inmediata                       
1   308926004  Si lo compraría de nuevo.                          
2   319828692  Muy bueno                                          
3   315254774  ممتاز                                              
4   318095277  Buenisimo                                          
5   321515119  Lo volveria a comprar                              
6   308797051  Si                                                 
7   308796996  Si                                                 
8   307387212  que se repita                                      
9   319285149  También                                            
10  314432978  Seguro que sí                                      
11  320946284  EXCELENTE                                          
12  308752702  Muy recomendable                                   
13  321393151  Very good                                      

In [80]:
from pyspark.sql.types import *

mySchema = StructType([StructField("UGC_ID", StringType(), True)\
,StructField("REVIEW_HEADLINE", StringType(), True)\
,StructField("REVIEW_COMMENTS", StringType(), True)\
,StructField("LANG", StringType(), True)\
,StructField("REVIEW_LENGTH", StringType(), True)\
,StructField("FL", IntegerType(), True)\
,StructField("ALL_TEXT", StringType(), True)\
,StructField("PRED_LANG", StringType(), True)\
,StructField("model_FL", IntegerType(), True)])



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [104]:
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

sfOptions = {
  'sfURL' : 'powerreviews.snowflakecomputing.com',
  'sfAccount' : 'powerreviews',
  'sfUser' : username,
  'sfPassword' : password,
  'sfDatabase' : 'ANALYTICS_WORKSHOP',
  'sfSchema' : 'DATA_SCIENCE',
  'sfWarehouse' : 'LOWER_ENV_WAREHOUSE'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [105]:
#Turn into spark dataframe in order to write to Snowflake
#Couldn't get write_pandas() to work to write directly from pandas to Snowflake
reviews = spark.createDataFrame(reviews, schema=mySchema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [106]:
reviews.write.format(SNOWFLAKE_SOURCE_NAME).options(**sfOptions) \
    .option("dbtable","FL_FL1_googletrans").mode("append").save()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [109]:
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TST_FP_TEST3")
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TST_FN_TEST3")
#conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TEST_NLP_SPARK6")
#conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TEST_NLP_SWAP_TOKENS")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…